An adversarial validation test, done at the beginning of a competition, can immediately reveal if train and test are similar or not. 

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Loading data 
X = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
X_test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
# Sampling
X = X.sample(n=200_000, random_state=0)
X_test = X_test.sample(n=200_000, random_state=0)

In [ ]:
# Preparing data as a tabular matrix
y = X.claim
X = X.set_index('id').drop('claim', axis='columns')
X_test = X_test.set_index('id')

In [ ]:
Xa = X.append(X_test)

mins = Xa.min() * 1.5
Xa = Xa.fillna(mins)
ya = [0] * len(X) + [1] * len(X_test)

In [ ]:
model = RandomForestClassifier()
cv_preds = cross_val_predict(model, Xa, ya, cv=5, n_jobs=-1, method='predict_proba')

In [ ]:
print(roc_auc_score(y_true=ya, y_score=cv_preds[:,1]))

In [ ]:
from matplotlib import pyplot as plt

plt.hist(cv_preds[:len(X), 1], bins=20, label='train')
plt.hist(cv_preds[len(X):, 1], bins=20, label='test')
plt.show()

In [ ]:
adversarial = pd.DataFrame({'id':X.index, 
                            'sim': cv_preds[:len(X), 1].ravel()})

adversarial.to_csv("adversarial_train.csv", index=False)

In [ ]:
adversarial = pd.DataFrame({'id':X_test.index, 
                            'sim': cv_preds[len(X):, 1].ravel()})

adversarial.to_csv("adversarial_test.csv", index=False)